# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

## Shift enter through initialize

In [6]:
import requests
import time
import json

In [7]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [8]:
class Queue():
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        if self.size() > 0:
            return self.queue.pop(0)
        else:
            return None
    def size(self):
        return len(self.queue)

In [9]:
# Import map and room info
with open('worldMap.json', 'r') as f:
    worldMap = json.load(f)
    
with open('worldRoomIndex.json', 'r') as f:
    worldRoomIndex = json.load(f)

In [10]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

## Input unique token key here

In [254]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/',
    "sell": 'adv/sell/',
    "drop": 'adv/drop/',
    "checkStatus": 'adv/status/',
    "examine": 'adv/examine/',
    "wearItem": 'adv/wear/',
    "removeItem": 'adv/undress/',
    "nameChange": 'adv/change_name/',
    "pray": 'adv/pray/',
    "fly": 'adv/fly/',
    "dash": 'adv/dash/',
    "companionLetCarry": 'adv/carry/',
    "companionReceiveBack":  'adv/receive/',
    "lambdaCoinProof": 'bc/last_proof/',
    "lambdaCoinMine": 'bc/mine/'
    
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']
sell = setupTable['url'] + setupTable['sell']
drop = setupTable['url'] + setupTable['drop']
checkStatus = setupTable['url'] + setupTable['checkStatus']
examine = setupTable['url'] + setupTable['examine']
wearItem = setupTable['url'] + setupTable['wearItem']
removeItem = setupTable['url'] + setupTable['removeItem']
nameChange = setupTable['url'] + setupTable['nameChange']
pray = setupTable['url'] + setupTable['pray']
fly = setupTable['url'] + setupTable['fly']
dash = setupTable['url'] + setupTable['dash']
companionLetCarry = setupTable['url'] + setupTable['companionLetCarry']
companionReceiveBack = setupTable['url'] + setupTable['companionReceiveBack']
lambdaCoinProof = setupTable['url'] + setupTable['lambdaCoinProof']
lambdaCoinMine = setupTable['url'] + setupTable['lambdaCoinMine']

In [325]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = worldMap
        self.coolDown = None
        self.currentRoomID = None
        self.previousRoomID = None
        self.lastMoveDirection = None
        self.currentRoomInfo = None
        
        self.status = None
        self.inventory = None
        
        self.lastProof = None
        
        self.validProof = None

        
    def getRoomInfo(self):
        time.sleep(1)
        response = requests.get(url=init, headers=header)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def assignCurrentRoomInfo(self):
        self.currentRoomInfo = self.getRoomInfo()
        
    def assignCurrentRoomID(self):
        self.currentRoomID = self.getCurrentRoomID()
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if str(self.currentRoomID) in self.playersMap and self.playersMap[str(self.currentRoomID)][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[str(self.currentRoomID)][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self):
        itemName = self.currentRoomInfo['items']
        for items in itemName:
            postRequestBody = {"name": items}
            response = requests.post(url=take, headers=header, json=postRequestBody)
            
            self.coolDown = response.json()['cooldown']
            print("Cooling down for :"+ str(self.coolDown)+"seconds")
            time.sleep(self.coolDown)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
    
    def sellItem(self):
        self.checkPlayerStatus()
        time.sleep(2)
        if self.inventory:
            for item in self.inventory:
                postRequestBody = {"name": item, "confirm":"yes"}
                response = requests.post(url=sell, headers=header, json=postRequestBody)
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                print("You have sold: ", item)
                
    def changeName(self):
        postRequestBody = {"name": "[CPHACHE]", "confirm":"aye"}
        response= requests.post(url=nameChange, headers=header, json=postRequestBody)
        
        print(response)
        print(response.json())
        
    def prayToShrine(self):
        response = requests.post(url=pray, headers=header)
        print(response.json())
        self.coolDown = response.json()['cooldown']
        time.sleep(self.coolDown)
        
    def getLambdaCoinProof(self):
        response = requests.get(url=lambdaCoinProof, headers=header)
        self.lastProof = response.json()
        return response.json()
    
    def mineCoin(self):
        newProof = self.validProof
        print(newProof)
        postRequestBody = {"name":newProof}
        response = requests.post(url=lambdaCoinMine, headers=header, json=postRequestBody)
        print(response.json())
        
    def examineWell(self):
        postRequestBody = {"name":"WELL"}
        response = requests.post(url=examine, headers=header, json=postRequestBody)
        return response.json()
        
    def checkPlayerStatus(self):
        response = requests.post(url=checkStatus, headers=header)
        self.status = response.json()
        self.coolDown = response.json()['cooldown']
        
        if response.json()['inventory']:
            self.inventory = response.json()['inventory']
            
        time.sleep(self.coolDown)
        
        print(response.json())
        
        

In [326]:
def exploreWorld(player):
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    count = 0
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1]      

            moveStack.pop()

    return roomDict          

In [327]:
def bfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathQueue = Queue()
    directionQueue = Queue()
    
    pathQueue.enqueue([str(starting_vertex)])
    directionQueue.enqueue(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathQueue.dequeue()
        directionPath = directionQueue.dequeue()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathQueue.enqueue(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionQueue.enqueue(directionCopy)

    return None

In [328]:
def dfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathStack = Stack()
    directionStack = Stack()
    
    pathStack.push([str(starting_vertex)])
    directionStack.push(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathStack.pop()
        directionPath = directionStack.pop()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathStack.push(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionStack.push(directionCopy)

    return None

In [329]:
def moveToSpecificRoom(player, starting_vertex, destination_vertex):
    path, directionPath = bfs(player, starting_vertex, destination_vertex)
    
    print(path)
    directions = directionPath[1:]
    print(directions)
    for direction in directions:
        player.moveRoom(direction)
        time.sleep(player.coolDown)
        
    print("You have reached your destination")

## Initialize Player

In [330]:
player = Player()
player.assignCurrentRoomInfo()
player.assignCurrentRoomID()

## World Points of Interest

Room 22 - Shrine
Room 461 - Shrine (checkpoint 125, 224)

## Everything below here is for exploration

In [263]:
player.getLambdaCoinProof()

{'proof': 40406328,
 'difficulty': 6,
 'cooldown': 1.0,
 'messages': [],
 'errors': []}

In [234]:
player.currentRoomInfo

{'room_id': 55,
 'title': 'Wishing Well',
 'description': "You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.",
 'coordinates': '(63,61)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['ken_ridenour',
  'Micah',
  'Labyrinth65',
  "['Dusty.Y']",
  'NP',
  '[JarredStanford]'],
 'items': [],
 'exits': ['w'],
 'cooldown': 7.5,
 'errors': [],
 'messages': ['You have walked east.', 'Wise Explorer: -50% CD']}

In [228]:
player.prayToShrine()

{'room_id': 499, 'title': "Glasowyn's Grave", 'description': "You see a moss-topped gravestone which reads: 'Here lies Glasowyn of Web17/Labs12/CS18, who was crushed under the weight of her own gold.' There doesn't seem to be any gold around, but marks in the dirt suggest that someone has knelt there, perhaps in prayer.", 'coordinates': '(51,67)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['e'], 'cooldown': 75.0, 'errors': [], 'messages': ['You sense a ghostly presence beside you, eager to ease your burden.']}


In [229]:
player.checkPlayerStatus()

{'name': '[CPHACHE]', 'cooldown': 1.0, 'encumbrance': 0, 'strength': 10, 'speed': 10, 'gold': 200, 'bodywear': None, 'footwear': None, 'inventory': [], 'abilities': ['pray', 'mine', 'fly', 'dash', 'carry'], 'status': [], 'has_mined': False, 'errors': [], 'messages': []}


In [ ]:
moveToSpecificRoom(player, player.currentRoomID, '200')

['0', '10', '19', '20', '27', '30', '32', '39']
['n', 'n', 'n', 'e', 'e', 'e', 'n']
Wise Explore
Wise Explore
Wise Explore
Wise Explore


In [149]:
player.moveRoom('n')

Wise Explore


{'room_id': 100,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(64,54)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': [],
 'exits': ['s', 'e', 'w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [175]:
player.changeName()

<Response [200]>
{'room_id': 467, 'title': "Pirate Ry's", 'description': "You see a sign before you that reads:\n\n'You have found Pirate Ry's. Send a `change_name` request and I'll change your identity to whatever you wish... for a price.'", 'coordinates': '(68,47)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['n'], 'cooldown': 30.0, 'errors': [], 'messages': ['You have changed your name to [CPHACHE].', "'Ere's a tip from Pirate Ry: If you find a shrine, try prayin'. Ye' never know who may be listenin'."]}


In [176]:
import pandas as pd

df = pd.DataFrame(worldRoomIndex)

In [180]:
df = df.T

In [182]:
df.title.value_counts()

A misty room                427
A Dark Cave                  36
Mt. Holloway                 29
A brightly lit room           1
Wishing Well                  1
The Transmogriphier           1
Linh's Shrine                 1
Pirate Ry's                   1
The Peak of Mt. Holloway      1
Shop                          1
Glasowyn's Grave              1
Name: title, dtype: int64

In [194]:
dft = df[df['title'] != 'A misty room']
dfo = dft[dft['title']!= 'A Dark Cave']
dff = dfo[dfo['title']!='Mt. Holloway']

In [224]:
for i in dff['description']:
    print(i)

You see a sign before you that reads:

'You have found Pirate Ry's. Send a `change_name` request and I'll change your identity to whatever you wish... for a price.'
You are standing at the zenith of Mt. Holloway. You see before you a holy shrine erected in the image of a magnificent winged deity.
You are standing in the center of a brightly lit room. You notice a shop to the west and exits to the north, south and east.
You are standing before a glowing shrine to a Linh, the Swift. She looks quite fast.
You see a moss-topped gravestone which reads: 'Here lies Glasowyn of Web17/Labs12/CS18, who was crushed under the weight of her own gold.' There doesn't seem to be any gold around, but marks in the dirt suggest that someone has knelt there, perhaps in prayer.
You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.
You are standing in a small shop. A sign behind the mechanical shopkeeper says 'WILL PAY FOR TREASURE'.
A strange machine stands in this 

## BlockChain

In [309]:
player.getLambdaCoinProof()

{'proof': 16314503,
 'difficulty': 6,
 'cooldown': 1.0,
 'messages': [],
 'errors': []}

In [319]:
player.getLambdaCoinProof()
proof_of_work(player)
player.mineCoin()

Searching for next proof
Proof found: 29721254 in 86.10790107499997
29721254
{'name': '[CPHACHE]', 'description': '[CPHACHE] looks like an ordinary person.', 'cooldown': 115.0, 'errors': ['There is no coin here: +100s'], 'messages': []}


In [320]:
import hashlib
import requests

import sys

from uuid import uuid4

from timeit import default_timer as timer

import random


def proof_of_work(player):
    start = timer()
    
    last_proof = player.lastProof['proof']
    
    
    print("Searching for next proof")
    #block_string = json.dumps(last_proof, sort_keys=True)
    proof = 0
    #  TODO: Your code here
    while valid_proof(last_proof, proof) is False:
        proof +=1
    print("Proof found: " + str(proof) + " in " + str(timer() - start))
    player.validProof = proof
    return proof


def valid_proof(last_hash, proof):
    
    difficulty = player.lastProof['difficulty']
    leadingZeros = '0'* difficulty
    
    guess = f'{last_hash}{proof}'.encode()
    guess_hash = hashlib.sha256(guess).hexdigest()
    if str(guess_hash)[:difficulty] == leadingZeros:
        print(guess_hash)
        return True
    else:
        return False


In [331]:
player.examineWell()

{'name': 'Wishing Well',
 'description': 'You see a faint pattern in the water...\n\n10000010\n00000001\n01001101\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n01100101\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01111001\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01110101\n01001000\n00000001\n10000010\n00000001\n01110010\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01100011\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000

In [332]:
print('\n\n10000010\n00000001\n01001101\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n01100101\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01111001\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01110101\n01001000\n00000001\n10000010\n00000001\n01110010\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01100011\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01110010\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01101101\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n00110011\n01001000\n00000001\n10000010\n00000001\n00111000\n01001000\n00000001\n10000010\n00000001\n00110101\n01001000\n00000001\n00000001')



10000010
00000001
01001101
01001000
00000001
10000010
00000001
01101001
01001000
00000001
10000010
00000001
01101110
01001000
00000001
10000010
00000001
01100101
01001000
00000001
10000010
00000001
00100000
01001000
00000001
10000010
00000001
01111001
01001000
00000001
10000010
00000001
01101111
01001000
00000001
10000010
00000001
01110101
01001000
00000001
10000010
00000001
01110010
01001000
00000001
10000010
00000001
00100000
01001000
00000001
10000010
00000001
01100011
01001000
00000001
10000010
00000001
01101111
01001000
00000001
10000010
00000001
01101001
01001000
00000001
10000010
00000001
01101110
01001000
00000001
10000010
00000001
00100000
01001000
00000001
10000010
00000001
01101001
01001000
00000001
10000010
00000001
01101110
01001000
00000001
10000010
00000001
00100000
01001000
00000001
10000010
00000001
01110010
01001000
00000001
10000010
00000001
01101111
01001000
00000001
10000010
00000001
01101111
01001000
00000001
10000010
00000001
01101101
01001000
00000001
10000010

In [322]:
player.getLambdaCoinProof()
proof_of_work(player)

Searching for next proof
00000008dd0aee9cbac6ee149035d13e41240f9c3b26b4260736838a73850031
Proof found: 32855342 in 91.50412111500009


32855342

In [324]:
player.mineCoin()

32855342
{'name': '[CPHACHE]', 'description': '[CPHACHE] looks like an ordinary person.', 'cooldown': 115.0, 'errors': ['There is no coin here: +100s'], 'messages': []}


In [ ]:
if __name__ == '__main__':
    # What node are we interacting with?
    if len(sys.argv) > 1:
        node = sys.argv[1]
    else:
        node = setupTable['url'] + setupTable['lambdaCoinMine']

    coins_mined = 0

    # Load or create ID
    f = open("my_id.txt", "r")
    id = f.read()
    print("ID is", id)
    f.close()

    if id == 'NONAME\n':
        print("ERROR: You must change your name in `my_id.txt`!")
        exit()
    # Run forever until interrupted
    while True:
        # Get the last proof from the server
        r = requests.get(url=node + "/last_proof")
        data = r.json()
        new_proof = proof_of_work(data.get('proof'))

        post_data = {"proof": new_proof,
                     "id": id}

        r = requests.post(url=node + "/mine", json=post_data)
        data = r.json()
        if data.get('message') == 'New Block Forged':
            coins_mined += 1
            print("Total coins mined: " + str(coins_mined))
        else:
            print(data.get('message'))